In [90]:
from senti_classifier import senti_classifier
from TwitterAPI import TwitterAPI
from datetime import datetime
import ConfigParser
import pickle
import nltk
import re

In [12]:
# Connect to the Twitter API and return a TwitterAPI object to use.
def get_twitter(config_file):
    config = ConfigParser.ConfigParser()
    config.read(config_file)
    twitter = TwitterAPI(
                   config.get('twitter', 'consumer_key'),
                   config.get('twitter', 'consumer_secret'),
                   config.get('twitter', 'access_token'),
                   config.get('twitter', 'access_token_secret'))
    return twitter

twitter = get_twitter('twitter.cfg')
print('Established Twitter connection.')

Established Twitter connection.


In [114]:
def process_tweet(tweet):
    #Convert to lower case
    tweet = tweet.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','AT_USER',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    
    return tweet


def gather_tweets(search_query, since, until):
    tweets = twitter.request('search/tweets',{'q':search_query, 'lang':'en', 'since':since, 'until':until, 'count':1})
    print tweets.json()['search_metadata']
    max_id = tweets.json()['search_metadata']['since_id_str']

    """
    for i in range(10):
            tweets = twitter.request('search/tweets',
                             {'q':search_query, 
                              'lang':'en', 
                              'since':since, 
                              'until':until,
                              'max_id':max_id,
                              'count':100})
            
            max_id = tweets.json()['search_metadata']['since_id_str']
            
            for tweet in tweets:
                process_text = process_tweet(tweet['text'])
                pos_score, neg_score = senti_classifier.polarity_scores(process_text)
                dico_tweets.append({'text': process_text, 
                                    'author': tweet['user']['name'], 
                                    'date': datetime.strptime(tweet['created_at'], '%a %b %d %H:%M:%S +0000 %Y').strftime('%Y-%m-%d'),
                                    'pos_score': pos_score,
                                    'neg_score': neg_score
                                   })
        
            print tweets.get_rest_quota()
            
            if not tweets.get_rest_quota():
                return dico_tweets                
    """

In [115]:
tweets_dic = gather_tweets('%24GOOG', '2015-11-23', '2015-11-30')

{u'count': 1, u'completed_in': 0.013, u'max_id_str': u'671115411778379777', u'since_id_str': u'0', u'next_results': u'?max_id=671115411778379776&q=%2524GOOG%20since%3A2015-11-23%20until%3A2015-11-30&lang=en&count=1&include_entities=1', u'refresh_url': u'?since_id=671115411778379777&q=%2524GOOG%20since%3A2015-11-23%20until%3A2015-11-30&lang=en&include_entities=1', u'since_id': 0, u'query': u'%2524GOOG+since%3A2015-11-23+until%3A2015-11-30', u'max_id': 671115411778379777}
